# CSV Writer - Comma-separated values

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://logiqx.github.io/gps-wizard/) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import time

import csv
from datetime import datetime

import unittest

from base_writer import BaseWriter

## Main Class

In [2]:
class CsvWriter(BaseWriter):
    '''CSV file - Comma-separated values'''

    def __init__(self, filename, tracks):
        '''Basic init just records the filename'''

        super().__init__(filename, tracks)


    def prepare(self, tracks):
        '''Prepare CSV prior to being saved'''

        for track in tracks:
            buffer = []

            # Basic header
            header = []
            for field in track.data:
                if field == 'ts':
                    header.append('date')
                    header.append('time')
                else:
                    header.append(field)

            buffer.append(header)

            # Determine field formats
            formats = self.getFormats(track)

            # Convert records to CSV
            for i in range(track.numPoints):
                record = []

                for field in track.data:
                    if field == 'ts':
                        dt = datetime.fromtimestamp(track.data[field][i])
                        record.append(dt.strftime('%Y-%m-%d'))
                        value = dt.strftime('%H:%M:%S.%f')[:-3]
                    elif field in formats:
                        value = formats[field].format(track.data[field][i])
                    else:
                        value = track.data[field][i]

                    record.append(value)

                buffer.append(record)

            self.buffers.append(buffer)


    def save(self):
        '''Save CSV file(s) to disk'''

        for i, buffer in enumerate(self.buffers):
            if len(self.buffers) > 1:
                parts = os.path.splitext(self.filename)
                filename = '{}-{}{}'.format(parts[0], i + 1, parts[1])
            else:
                filename = self.filename

            with open(filename, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(self.buffers[i])

## Unit Tests

In [3]:
class TestCsvWriter(unittest.TestCase):
    '''Class to test the CSV writer'''

    def testCsvWriter(self):
        '''Test the CSV writer'''

        filename = os.path.join(projdir, 'sessions', 'unittest.csv')

        csvWriter = CsvWriter(filename, sbnReader.tracks)

        csvWriter.save()
        
        os.unlink(filename)

In [4]:
if __name__ == '__main__':
    for path in ['python', '.', '..']:
        readersPath = os.path.join(path, 'core')
        if readersPath not in sys.path:
            sys.path.extend([readersPath])

    from file_reader import getFileReader

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    sbnFilename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_165512_DLG.SBP')
    sbnReader = getFileReader(sbnFilename)

    pc1 = time.perf_counter()
    sbnReader.load()
    pc2 = time.perf_counter()

    print("\nTest file loaded in %0.2f seconds" % (pc2 - pc1))


Test file loaded in 0.02 seconds


In [5]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.
----------------------------------------------------------------------
Ran 1 test in 0.044s

OK
